![Practicum AI Logo image](https://github.com/PracticumAI/practicumai.github.io/blob/main/images/logo/PracticumAI_logo_250x50.png?raw=true)
***
# *Practicum AI:* RNN - Simple RNN

This exercise was adapted from Baig et al. (2020) <i>The Deep Learning Workshop</i> from <a href="https://www.packtpub.com/product/the-deep-learning-workshop/9781839219856">Packt Publishers</a> (Exercise 5.03, page 243)

In this exercise, we will build a simple RNN model using Keras. The model will consist of a reshaped layer followed by a SimpleRNN layer, and a dense layer for prediction. We will use the formatted trainX and trainY data, as well as initialized layers from Keras.

<div style="padding: 10px;margin-bottom: 20px;border: thin solid #30335D;border-left-width: 10px;background-color: #fff"><strong>Note:</strong> The setup code below comes from the data visualize exercise and must be run prior to this one. </div>

In [2]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
inp0 = pd.read_csv('data/AAPL.csv')
inp0.head()

,Date,Close,Open,High,Low,Volume
0,1/17/2020,138.31,136.54,138.330,136.16,5623336
1,1/16/2020,137.98,137.32,138.190,137.01,4320911
2,1/15/2020,136.62,136.00,138.055,135.71,4045952
3,1/14/2020,135.82,136.28,137.139,135.55,3683458
4,1/13/2020,136.60,135.48,136.640,135.07,3531572


In [20]:
ts_data = inp0.Close.values.reshape(-1,1)

#### Preparing the Data for Stock Price Prediction

In exercise `01.1_data_visualize`, we visualized the dataset.  Now, we need to prepare the data for the model. 

**Step 1,  create a train_test split of the data.** 

With time-series data, we can't simply select random data points for our training and testing sets. Instead, we must maintain the sequence of the data. A common approach for handling time-series data is to use the first portion of the data for training and the last portion for testing. In this case, we will use the first 75% of the records as the training data and the last 25% as the test data. It is important to preserve the temporal order of the data when working with time-series data.

In [5]:
train_recs = int(len(ts_data) * 0.75)

In [6]:
train_data = ts_data[:train_recs]
test_data = ts_data[train_recs:]
len(train_data), len(test_data)

(1885, 629)

**Step 2, scale the stock data.**

To scale the data so values are between 0 and 1 (inclusive), we use the `MinMaxScaler` from sklearn. This scaler maps the highest value in the data to 1. We then fit and transform the scaler on the training data but only transform the test data. This ensures that the scaling parameters learned from the training data are applied to the test data, allowing for a realistic evaluation of the model's performance.

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)

**Step 3, format the data to get "features" for each instance.**

To predict the next value, we need to define a *lookback period*, which is the number of previous days' data that we want to use as input. We can define a function that returns the target value (in this case, the stock price for a particular day) and the daily values in the *lookback period*. This function can be used to create training and testing sets that include the appropriate number of input days and corresponding target values.

In [8]:
def get_lookback(inp, look_back):
    y = pd.DataFrame(inp)
    dataX = [y.shift(i) for i in range(1, look_back + 1)]
    dataX = pd.concat(dataX, axis = 1)
    dataX.fillna(0, inplace = True)
    
    return dataX.values, y.values

Now, we define a lookback period and apply the function to our data.

In [9]:
look_back = 10
trainX, trainY = get_lookback(train_scaled, look_back = look_back)
testX, testY = get_lookback(test_scaled, look_back = look_back)

In [10]:
trainX.shape, testX.shape

((1885, 10), (629, 10))

As expected, each example includes 10 features, corresponding to the past 10 days of data. We have this historical data available for both the training and testing sets.

Now, we can move on to building and training a model using this prepared data.

#### Exercise 5.03 (Student)

***

#### 1. Import the requisite libraries

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers \
import SimpleRNN, Activation, Dropout, Dense, Reshape
```

In [1]:
# Code it!

#### 2. Instantiate a sequential model

```python
model = Sequential()
```

In [11]:
# Code it!

#### 3. Add a reshape layer

```python
model.add(Reshape((look_back,1), input_shape = (look_back,)))
```

In [12]:
# Code it!

#### 4. Add a simple RNN layer with 32 neurons and specify input shape

```python
model.add(SimpleRNN(32, input_shape=(look_back, 1)))
```

In [13]:
# Code it!

#### 5. Add a dense layer

```python
model.add(Dense(1))
```

In [14]:
# Code it!

#### 6. Add an activation layer with a linear function

```python
model.add(Activation('linear'))
```

In [15]:
# Code it!

#### 7. Compile the model with an Adam optimizer

```python
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
```

In [1]:
# Code it!

#### 8. Print a summary of the model

```python
model.summary()
```

In [ ]:
# Code it!

The architecture of our single-layer (plain) RNN model is now complete.  This is a relatively simple model compared to those we have built for image data in the past.  But how will it perform?

Let's see how it does with our current task. By evaluating the model's performance, we can determine whether this simple architecture is sufficient or if we need to consider a more complex model.

***
####  Model Training and Performance Evaluation

It's time to fit the model to the data.  To do that, we'll set batch size to 1, validation split to 10%, and train the model for three epochs. Experiment with the number of epochs to see which produces the best results.

Train the model using the `fit()` method.

```python
model.fit(trainX, trainY, epochs = 3, batch_size = 1, verbose = 2, validation_split = 0.1)
```

In [2]:
# Code it!

The loss appears to be quite low. With training complete, we need to evaluate the model's performance on the training and testing sets. To do this, we define two functions: one to print the root mean squared error (RMS) on the training and testing sets, and another to plot the predictions for the test data alongside the original values in the data.  The RMS error and prediction visualization should provide valuable insights into the model's accuracy and reliability.

In [ ]:
import math
def get_model_perf(model_obj):
    score_train = model_obj.evaluate(trainX, trainY, verbose=0)
    print('Train RMSE: %.2f RMSE' % (math.sqrt(score_train)))
    score_test = model_obj.evaluate(testX, testY, verbose=0)
    print('Test RMSE: %.2f RMSE' % (math.sqrt(score_test)))

In [ ]:
get_model_perf(model)

The RMS error values appear to be relatively low.  Even so, we probably ought to visually assess the model's performance by comparing the actual values to the predicted values for the test period.  Doing so will give us a more intuitive understanding of how well the model predicts stock prices.  By plotting actual vs predicted values on the same graph, we can quickly identify discrepancies and see whether the model is making reasonable predictions.   

In [ ]:
def plot_pred(model_obj):
    testPredict = scaler.inverse_transform(model_obj.predict(testX))
    
    pred_test_plot = ts_data.copy()
    pred_test_plot[:train_recs+look_back,:] = np.nan
    pred_test_plot[train_recs+look_back:,:] = testPredict[look_back:]
    
    plt.plot(ts_data)
    plt.plot(pred_test_plot, "--")

This function first makes predictions on the test data. Since our data has been scaled, we need to apply the inverse transform to get it back to its original scale before plotting it. The function then plots actual values as a solid line and predicted values as dotted lines. 

In [ ]:
%matplotlib inline
plt.figure(figsize=[10,5])
plot_pred(model)

The plot visualizes the predictions made by the model (dotted lines) alongside the actual values (solid lines). At first glance, the model appears to be doing a good job.  But to get a better understanding of the data, let's zoom in on a specific section of the plot to see individual points more clearly. 

To focus on a specific range of plot indices, we can use the `xlim` function to set the x-axis limits of the plot. For example, to focus on indices 2300-2500, you can use the following code:

In [ ]:
%matplotlib inline
plot_pred(model)
plt.xlim(2300, 2500) # set the x-axis limits
plt.ylim(120, 150)   # set the y-axis limits

After zooming in, the results look good. The model has captured most of the variations in the data. It's impressive that a single RNN layer with just 32 neurons achieves does so well.  Indeed, you might be pleasantly surprised by the efficacy of RNNs on this task.  The ability of RNNs to process sequential data and capture temporal dependencies makes them well-suited for time series prediction tasks.

#### Summary

In this exercise, we constructed a basic RNN model to predict stock prices. We also saw that even a simple model can have strong predictive power with sequence prediction tasks. By training the model on historical stock data, we were able to make relatively accurate predictions about future prices. This demonstrates the potential of RNNs for time series prediction tasks and the importance of considering temporal dependencies when working with sequential data.